# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [72]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time 

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

# Output File (CSV)
output_data_file = "../output_data/city_weather.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [73]:
vacay_df = pd.read_csv(output_data_file)
vacay_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,souillac,75,MU,1619395665,88,-20.5167,57.5167,77.00,14.97
1,vaini,75,TO,1619395666,83,-21.2000,-175.2000,80.60,6.91
2,ushuaia,75,AR,1619395666,71,-54.8000,-68.3000,48.20,20.71
3,nanortalik,100,GL,1619395667,85,60.1432,-45.2371,39.25,7.70
4,cherskiy,95,RU,1619395667,83,68.7500,161.3000,17.42,6.22
...,...,...,...,...,...,...,...,...,...
541,lutkun,84,RU,1619396197,65,41.4808,47.6858,56.89,8.95
542,cayenne,75,GF,1619396197,83,4.9333,-52.3333,81.00,3.44
543,saint-joseph,0,RE,1619396198,73,-21.3667,55.6167,77.00,16.11
544,mitsamiouli,81,KM,1619396198,76,-11.3847,43.2844,80.53,6.67


In [74]:
# Drop NaN values
vacay_df = vacay_df.dropna()
vacay_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,souillac,75,MU,1619395665,88,-20.5167,57.5167,77.00,14.97
1,vaini,75,TO,1619395666,83,-21.2000,-175.2000,80.60,6.91
2,ushuaia,75,AR,1619395666,71,-54.8000,-68.3000,48.20,20.71
3,nanortalik,100,GL,1619395667,85,60.1432,-45.2371,39.25,7.70
4,cherskiy,95,RU,1619395667,83,68.7500,161.3000,17.42,6.22
...,...,...,...,...,...,...,...,...,...
541,lutkun,84,RU,1619396197,65,41.4808,47.6858,56.89,8.95
542,cayenne,75,GF,1619396197,83,4.9333,-52.3333,81.00,3.44
543,saint-joseph,0,RE,1619396198,73,-21.3667,55.6167,77.00,16.11
544,mitsamiouli,81,KM,1619396198,76,-11.3847,43.2844,80.53,6.67


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [89]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = vacay_df[["Lat", "Lng"]]

# Add humidity heat map layer

humidity = vacay_df["Humidity"]

# Create a humidity Heatmap layer
fig = gmaps.figure(center=(48.0, -7.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

#### A max temperature lower than 85 degrees but higher than 70.

#### Wind speed less than 10 mph.

#### Zero cloudiness.

#### Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [90]:
vacay_locale_df = vacay_df.loc[(vacay_df["Max Temp"].between(70, 85, inclusive=True)) & 
                              (vacay_df["Wind Speed"] < 10) & (vacay_df["Cloudiness"] == 0)].dropna()

vacay_locale_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,atar,0,MR,1619395668,19,20.5169,-13.0499,78.49,3.69
17,karratha,0,AU,1619395531,42,-20.7377,116.8463,78.89,9.10
29,roebourne,0,AU,1619395677,40,-20.7833,117.1333,78.84,9.98
96,namibe,0,AO,1619395734,67,-15.1961,12.1522,76.57,9.75
119,beloha,0,MG,1619395773,74,-25.1667,45.0500,71.06,6.67
298,abu dhabi,0,AE,1619395944,50,24.4667,54.3667,84.20,1.14
331,kashmor,0,PK,1619395989,14,28.4333,69.5833,84.65,5.73
346,hurghada,0,EG,1619395997,60,27.2574,33.8129,73.40,1.90
405,araxa,0,BR,1619395874,43,-19.5933,-46.9406,73.40,5.75
486,baherden,0,TM,1619396148,29,38.4362,57.4316,74.64,5.84


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [91]:
# Store hotel map into variable 
hotel_df = pd.DataFrame(vacay_locale_df, columns = ["City", "Country", "Lat", "Lng"])
# Add a column named "Hotel Name"
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
7,atar,MR,20.5169,-13.0499,
17,karratha,AU,-20.7377,116.8463,
29,roebourne,AU,-20.7833,117.1333,
96,namibe,AO,-15.1961,12.1522,
119,beloha,MG,-25.1667,45.0500,
298,abu dhabi,AE,24.4667,54.3667,
331,kashmor,PK,28.4333,69.5833,
346,hurghada,EG,27.2574,33.8129,
405,araxa,BR,-19.5933,-46.9406,
486,baherden,TM,38.4362,57.4316,


In [92]:
# set up a parameters dictionary with hotels within 5000 meters of nearest city
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [93]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    vacay_city = row["City"]
    
    params["location"] = f"{lat}, {lng}"
    
    print(f"Retrieving Results for Index {index}: {vacay_city}.")
       
    hotel_response = requests.get(base_url, params=params).json()
    
    hotel_results = hotel_response["results"]
    
    try:
        print(f"Closest hotel in {vacay_city} is {hotel_results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_name.append(np.nan)
        print("Could not find result")
        print("-----------------------------")
        time.sleep(5)
        print("Ending data retrieval")

Retrieving Results for Index 7: atar.
Closest hotel in atar is Bab Sahara.
Retrieving Results for Index 17: karratha.
Closest hotel in karratha is Karratha International Hotel.
Retrieving Results for Index 29: roebourne.
Closest hotel in roebourne is Latitude20 Roebourne Village.
Retrieving Results for Index 96: namibe.
Closest hotel in namibe is Hotel Chik Chik Namibe.
Retrieving Results for Index 119: beloha.
Could not find result
-----------------------------
Ending data retrieval
Retrieving Results for Index 298: abu dhabi.
Closest hotel in abu dhabi is Emirates Palace.
Retrieving Results for Index 331: kashmor.
Closest hotel in kashmor is Gul Muhammad Hotel.
Retrieving Results for Index 346: hurghada.
Closest hotel in hurghada is Albatros White Beach - All Inclusive.
Retrieving Results for Index 405: araxa.
Closest hotel in araxa is Hotel Dona Beja.
Retrieving Results for Index 486: baherden.
Could not find result
-----------------------------
Ending data retrieval


In [94]:
hotel_df.dropna()
hotel_df.to_csv("../output_data/vacay_hotel.csv", header=True)

hotel_df

,City,Country,Lat,Lng,Hotel Name
7,atar,MR,20.5169,-13.0499,Bab Sahara
17,karratha,AU,-20.7377,116.8463,Karratha International Hotel
29,roebourne,AU,-20.7833,117.1333,Latitude20 Roebourne Village
96,namibe,AO,-15.1961,12.1522,Hotel Chik Chik Namibe
119,beloha,MG,-25.1667,45.0500,
298,abu dhabi,AE,24.4667,54.3667,Emirates Palace
331,kashmor,PK,28.4333,69.5833,Gul Muhammad Hotel
346,hurghada,EG,27.2574,33.8129,Albatros White Beach - All Inclusive
405,araxa,BR,-19.5933,-46.9406,Hotel Dona Beja
486,baherden,TM,38.4362,57.4316,


In [95]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [96]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations)
fig.add_layer(marker)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))